# Tekstclassificatie

Een veel voorkomende toepassing bij machine learning is het werken op tekstfragmenten.
Denk bijvoorbeeld aan chatbots, spam-mail detectie, automatische call-centers, ...
Dit wordt ook NLP of Natural Language Processing genoemd.
De moeilijkheid bij het werken met NLP is dat de betekenis van een woord vaak sterk afhankelijk is van de context van het stuk tekst.
Om met deze complexiteit om te gaan zijn geavanceerde machine learning technieken nodig zoals deep learning.
Dit komt later bij Machine Learning in meer detail aan bod.
De complexiteit kan echter sterk gereduceerd worden door te veronderstellen dat alle woorden in het tekstfragment onafhankelijk zijn van elkaar.
In de rest van deze notebook wordt gewerkt met het typische voorbeeld over spam-detectie op inkomende mails.

## Typisch voorbeeld: Spam detectie

Hieronder staat een typisch voorbeeld van een spam mail.

![spam-example](images\spam1.jpg)

Het doel is nu om deze mail te classificeren als spam of not spam (dit laatste wordt ook "ham" genoemd).
We zoeken dus de kans dat deze tekst spam is op basis van alle woorden die erin staat.
Hierdoor bekomen we de volgende vergelijking.

$P(\text{Spam}|w_1,w_2, \dots w_n) = \frac{P(w_1,w_2, \dots w_n|\text{Spam}) P(\text{Spam})}{P(w_1,w_2, \dots w_n)}$

Na toepassing van de Bayes rule kunnen we dit ook schrijven als:

$P(\text{Spam}|w_1,w_2, \dots w_n) = \frac{P(w_1|w_2, \dots w_n,\text{Spam})P(w_2|w_3, \dots w_n,\text{Spam})\dots P(\text{Spam})}{P(w_1,w_2, \dots w_n)}$

De kansen in de teller van de breuk geven de kansen weer om een bepaald woord te vinden op basis van de andere woorden in de tekst.
Dit is heel lastig te berekenen en zorgt voor een heel hoge complexiteit.
Hier komt echter de veronderstelling van onafhankelijkheid van de woorden in de tekst goed van pas om deze vergelijking te vereenvoudigen.
We bekomen namelijk:

$P(\text{Spam}|w_1,w_2, \dots w_n) = \frac{P(w_1|\text{Spam})P(w_2|\text{Spam})\dots P(w_n|\text{Spam})P(\text{Spam})}{P(w_1,w_2, \dots w_n)} = \frac{P(\text{Spam})\prod \limits_{i=1}^{n}P(w_i|\text{Spam})}{P(w_1,w_2, \dots w_n)}$

De noemer hierin zorgt ervoor dat de teller terug omgezet wordt naar een kans (met een waarde tussen 0 en 1).
Deze is echter onafhankelijk van de kans of het spam is of niet.
We hebben ook niet de exacte kans nodig maar moeten gewoon weten of de teller het grootst is als het spam is of niet.
Daarom kan de noemer dus weggelaten worden met als volgende resultaat.

$P(\text{Spam}|w_1,w_2, \dots w_n) \propto P(\text{Spam})\prod \limits_{i=1}^{n}P(w_i|\text{Spam})$

Stel dat we de volgende gegevens hebben over een dataset met 300 spam mails en 850 ham mails:

In [1]:
import pandas as pd

In [2]:
aantal_spam_mails = 300
aantal_ham_mails = 850

woord = ["customer", "advise", "Africa", "money", "number"]
spam_freq = [100,50, 120,60, 180]
spam_prob = [x/aantal_spam_mails for x in spam_freq]
ham_freq = [200, 70, 30, 450, 660]
ham_prob =  [x/aantal_ham_mails for x in ham_freq]

df = pd.DataFrame({"Woord": woord, "Spam frequentie": spam_freq, "Spam kans": spam_prob, "Ham frequentie": ham_freq, "Ham kans": ham_prob})
df

,Woord,Spam frequentie,Spam kans,Ham frequentie,Ham kans
0,customer,100,0.333333,200,0.235294
1,advise,50,0.166667,70,0.082353
2,Africa,120,0.400000,30,0.035294
3,money,60,0.200000,450,0.529412
4,number,180,0.600000,660,0.776471


Beantwoord nu de volgende vragen:
* Wat is $P(\text{Spam})$, de kans dat een willekeurige mail spam is in de dataset?
* Doe een manuele classificatie van de zin "Africa advise money"

In [3]:
pSpam = aantal_spam_mails / (aantal_ham_mails+aantal_spam_mails)
pHam = 1 - pSpam

pTextIsSpam = pSpam * 0.4 * 0.1666 * 0.2
pTextIsHam = pHam * 0.035 * 0.082 * 0.529 

if(pTextIsSpam > pTextIsHam):
    print("Africa advise money is spam")
else:
    print("Africa advise money is ham")

Africa advise money is spam


**Hoe wordt er omgegaan met classificatie van woorden die niet in de dataset zaten?**

Indien een mail uit de testset een woord bevat die nooit gezien was in de trainingsset, dan kan deze mail niet geclassificeerd worden omdat de kans van het woord steeds 0 is.
Er zijn twee mogelijkheden om hiermee om te gaan. 
* Ofwel laat je die eruit vallen maar dan verlies je informatie.
* Ofwel geef je deze woorden toch een bepaalde kans.

Dit laatste kan gebeuren door middel van Laplacian Smoothing.
De formule hiervoor is: 

$P(w) = \frac{C(w) + \alpha}{N+\alpha V}$

met 
* P(w) de uiteindelijke kans van het woord
* C(w) het aantal mails waarin het woord voorkomt
* N het aantal spam-mails
* V het aantal verschillende woorden (features) in de dataset
* $\alpha$ een hyperparameter dat aangeeft hoeveel gewicht de ongeziene woorden moeten krijgen.
    * Kleine waarde $\rightarrow$ klein belang van ongeziene woorden $\rightarrow$ neiging tot overfitting
    * Grote waarde $\rightarrow$ groot belang van ongeziene woorden $\rightarrow$ neiging tot underfitting 
    
Het algemene concept van Laplacian smoothing is om voor ongeziene woorden een extra fictieve mail toe te voegen die enkel bestaat uit dit woord.
De $\alpha$ is dan hoeveel keer deze mail wordt toegevoegd.
Bereken voor een $\alpha=2$ opnieuw de kansen in het dataframe df.
Voeg hier ook een lijn aan toe voor woorden die niet in de dataset aanwezig zijn.

In [4]:
# bereken de geupdate matrix met de kansen

alpha = 2
V = len(df)

df["pSpamUnknown"] = (df["Spam frequentie"] + alpha) / (aantal_spam_mails + alpha * V)
df["pHamUnknown"] = (df["Ham frequentie"] + alpha) / (aantal_ham_mails + alpha * V)

p_spam_unknown =  (alpha) / (aantal_spam_mails + alpha * V)
p_ham_unknown =  (alpha) / (aantal_ham_mails + alpha * V)

df

,Woord,Spam frequentie,Spam kans,Ham frequentie,Ham kans,pSpamUnknown,pHamUnknown
0,customer,100,0.333333,200,0.235294,0.329032,0.234884
1,advise,50,0.166667,70,0.082353,0.167742,0.083721
2,Africa,120,0.400000,30,0.035294,0.393548,0.037209
3,money,60,0.200000,450,0.529412,0.200000,0.525581
4,number,180,0.600000,660,0.776471,0.587097,0.769767


Voer nu opnieuw manuele classificatie uit voor de zin "Europe advise money" door gebruik te maken van Laplacian Smoothing.
Is deze zin Spam of Ham?

In [5]:
pTextIsSpam = pSpam * p_spam_unknown * 0.167742 * 0.2
pTextIsHam = pHam * p_ham_unknown * 0.083721 * 0.525581

if(pTextIsSpam > pTextIsHam):
    print("Europe advise money is spam")
else:
    print("Europe advise money is ham")

Europe advise money is ham


In [6]:
print(pTextIsSpam, pTextIsHam)

5.646294530154278e-05 7.563567616956521e-05


**Probleem van veel kansen te vermenigvuldigen**

Naast het probleem van ongeziene woorden, kan er nog een probleem optreden bij het werken met de kansberekeningen.
Aangezien kansen een waarde tussen 0 en 1 zijn kan het zijn dat er bij vermenigvuldiging van veel kansen een floating-point underflow optreed omdat het resultaat steeds kleiner en kleiner gaat worden.
Om dit tegen te gaan kan men gebruik maken van het logaritme van de kansen.
Dit wordt ook **log likelihood** genoemd en wordt berekend als volgt:

$log(P(\text{Spam}|w_1,w_2, \dots w_n)) \propto log(P(\text{Spam})) + \sum\limits_{i=1}^{n}log(P(w_i|\text{Spam}))$

De resulterende klasse is nog steeds de klasse met de hoogste log-likelihood.

Bereken de log-likelihood voor de zin "Europe advice money".

In [7]:
import math

pTextIsSpam = math.log(pSpam) + math.log(p_spam_unknown) + math.log(0.167742) + math.log(0.2)
pTextIsHam = math.log(pHam) + math.log(p_ham_unknown) + math.log(0.083721) + math.log(0.525581)

if(pTextIsSpam > pTextIsHam):
    print("Europe advise money is spam")
else:
    print("Europe advise money is ham")

print(pTextIsSpam, pTextIsHam)  # de logartime behoudt de vergelijking tussen de twee waarden

Europe advise money is ham
-9.781925970336896 -9.489582479087048


**Probleem van gelijkaardige woorden**

Daarnaast is het duidelijk dat elke vorm van geschreven tekst een groot aantal overbodige woorden betekenen. 
Dit zijn dan bijvoorbeeld woorden die in heel veel zinnen voorkomen zoals ik, hij, en, daar, ...
Deze woorden gaan geen informatie geven om de classificatie uit te voeren en kunnen dus ook genegeerd worden.
Daarnaast kan men ook de vraag stellen over vervoegingen van werkwoorden of meervouden een apart woord moeten zijn.

Over het algemeen gezien kunnen we de volgende stappen uitvoeren om de tekst bruikbaarder te maken:
* Html/xml/... tags verwijderen indien er zijn ([BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/))
* Cijfers/speciale symbolen verwijderen
* Alles omzetten naar lowercase
* Stopwoorden verwijderen ([nltk.corpus](https://www.geeksforgeeks.org/removing-stop-words-nltk-python/)) (vergeet niet de stopwords te downloaden indien nodig)
* Alle woorden herleiden naar hun stam ([nltk SnowballStemmer](https://www.nltk.org/_modules/nltk/stem/snowball.html))
* Verwijder te korte woorden

Deze stappen kunnen door middel van de volgende lijnen code uitgevoerd worden:

In [12]:
import opendatasets as od
od.download("https://www.kaggle.com/chandramoulinaidu/spam-classification-for-basic-nlp")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading spam-classification-for-basic-nlp.zip to .\spam-classification-for-basic-nlp


100%|██████████| 5.69M/5.69M [00:00<00:00, 16.1MB/s]


In [14]:
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 11.9 MB/s eta 0:00:00
  Using cached regex-2022.10.31-cp310-cp310-win_amd64.whl (267 kB)


In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

# load dataset
df = pd.read_csv(".\\spam-classification-for-basic-nlp\\Spam Email raw text for NLP.csv")
display(df.head(10))

# remove html
df.MESSAGE = df.MESSAGE.apply(lambda text: BeautifulSoup(text, 'html.parser').get_text())
display(df.head(10))

# remove non-letters
df.MESSAGE = df.MESSAGE.apply(lambda text: re.sub("[^a-zA-Z’]", " ", str(text)))
display(df.head(10))

# to lower-case
df.MESSAGE = df.MESSAGE.apply(lambda text: text.lower())
display(df.head(10))

# remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    txt = ""
    for w in word_tokens:
        if w not in stop_words:
            txt += w + " "
    return txt

df.MESSAGE = df.MESSAGE.apply(remove_stopwords)
display(df.head(10))

# Herleiden tot de stam
def stemmer(text):
    stemmer = SnowballStemmer('english')
    
    word_tokens = word_tokenize(text)
    txt = ""
    for w in word_tokens:
        txt += stemmer.stem(w) + " "
    return txt

df.MESSAGE = df.MESSAGE.apply(stemmer)
display(df.head(10))

# Remove short words
def remove_short_words(text):
    word_tokens = word_tokenize(text)
    minWordSize = 5
    txt = ""
    for w in word_tokens:
        if len(w) >= minWordSize:
            txt += w + " "
    return txt
    
df.MESSAGE = df.MESSAGE.apply(remove_short_words)
display(df.head(10))

,CATEGORY,MESSAGE,FILE_NAME
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,ATTENTION: This is a MUST for ALL Computer Use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,This is a multi-part message in MIME format.\n...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,00210.050ffd105bd4e006771ee63cabc59978
4,1,This is the bottom line. If you can GIVE AWAY...,00033.9babb58d9298daa2963d4f514193d7d6
5,1,------=_NextPart_000_00B8_51E06B6A.C8586B31\n\...,00187.efd97ab2034b3384606e21db00014ecb
6,1,"<STYLE type=""text/css"">\n\n<!--\n\nP{\n\n fon...",00500.85b72f09f6778a085dc8b6821965a76f
7,1,<HR>\n\n<html>\n\n<head>\n\n <title>Secured I...,00493.1c5f59825f7a246187c137614fb1ea82
8,1,"<table width=""600"" border=""20"" align=""center"" ...",00012.381e4f512915109ba1e0853a7a8407b2
9,1,"<html>\n\n\n\n<head>\n\n<meta http-equiv=""Cont...",00487.139a2f4e8edbbdd64441536308169d74


c:\Users\jens.baetens3\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
c:\Users\jens.baetens3\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


,CATEGORY,MESSAGE,FILE_NAME
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,ATTENTION: This is a MUST for ALL Computer Use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,This is a multi-part message in MIME format.\n...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,00210.050ffd105bd4e006771ee63cabc59978
4,1,This is the bottom line. If you can GIVE AWAY...,00033.9babb58d9298daa2963d4f514193d7d6
5,1,------=_NextPart_000_00B8_51E06B6A.C8586B31\n\...,00187.efd97ab2034b3384606e21db00014ecb
6,1,"\n\n\n(Hello,This is Chinese Traditional)\n\n\...",00500.85b72f09f6778a085dc8b6821965a76f
7,1,\n\n\nSecured Investements\n\n \n\n\n\n\n\n\n...,00493.1c5f59825f7a246187c137614fb1ea82
8,1,\n\n\n\nClick Here Now !\nSimply Amateur\nJust...,00012.381e4f512915109ba1e0853a7a8407b2
9,1,\n\n\n\n\n\nAnswer-Us\n\n\n \n\n\n\n\n\n\n\n\n...,00487.139a2f4e8edbbdd64441536308169d74


,CATEGORY,MESSAGE,FILE_NAME
0,1,Dear Homeowner Interest Rates are at thei...,00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,ATTENTION This is a MUST for ALL Computer Use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,This is a multi part message in MIME format ...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,IMPORTANT INFORMATION The new domain names...,00210.050ffd105bd4e006771ee63cabc59978
4,1,This is the bottom line If you can GIVE AWAY...,00033.9babb58d9298daa2963d4f514193d7d6
5,1,NextPart B E B A C B C...,00187.efd97ab2034b3384606e21db00014ecb
6,1,Hello This is Chinese Traditional ...,00500.85b72f09f6778a085dc8b6821965a76f
7,1,Secured Investements WEALTH WITH...,00493.1c5f59825f7a246187c137614fb1ea82
8,1,Click Here Now Simply Amateur Just like ...,00012.381e4f512915109ba1e0853a7a8407b2
9,1,Answer Us Unlist Info...,00487.139a2f4e8edbbdd64441536308169d74


,CATEGORY,MESSAGE,FILE_NAME
0,1,dear homeowner interest rates are at thei...,00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,attention this is a must for all computer use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,this is a multi part message in mime format ...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,important information the new domain names...,00210.050ffd105bd4e006771ee63cabc59978
4,1,this is the bottom line if you can give away...,00033.9babb58d9298daa2963d4f514193d7d6
5,1,nextpart b e b a c b c...,00187.efd97ab2034b3384606e21db00014ecb
6,1,hello this is chinese traditional ...,00500.85b72f09f6778a085dc8b6821965a76f
7,1,secured investements wealth with...,00493.1c5f59825f7a246187c137614fb1ea82
8,1,click here now simply amateur just like ...,00012.381e4f512915109ba1e0853a7a8407b2
9,1,answer us unlist info...,00487.139a2f4e8edbbdd64441536308169d74


,CATEGORY,MESSAGE,FILE_NAME
0,1,dear homeowner interest rates lowest point yea...,00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,attention must computer users new special pack...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,multi part message mime format nextpart cdc c ...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,important information new domain names finally...,00210.050ffd105bd4e006771ee63cabc59978
4,1,bottom line give away cd free people like one ...,00033.9babb58d9298daa2963d4f514193d7d6
5,1,nextpart b e b c b content type text plain cha...,00187.efd97ab2034b3384606e21db00014ecb
6,1,hello chinese traditional f r v c w n n e mail...,00500.85b72f09f6778a085dc8b6821965a76f
7,1,secured investements wealth without risk disco...,00493.1c5f59825f7a246187c137614fb1ea82
8,1,click simply amateur like girl next door xxx f...,00012.381e4f512915109ba1e0853a7a8407b2
9,1,answer us unlist information message brought a...,00487.139a2f4e8edbbdd64441536308169d74


,CATEGORY,MESSAGE,FILE_NAME
0,1,dear homeown interest rate lowest point year h...,00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,attent must comput user new special packag dea...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,multi part messag mime format nextpart cdc c b...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,import inform new domain name final avail gene...,00210.050ffd105bd4e006771ee63cabc59978
4,1,bottom line give away cd free peopl like one m...,00033.9babb58d9298daa2963d4f514193d7d6
5,1,nextpart b e b c b content type text plain cha...,00187.efd97ab2034b3384606e21db00014ecb
6,1,hello chines tradit f r v c w n n e mail w n c...,00500.85b72f09f6778a085dc8b6821965a76f
7,1,secur invest wealth without risk discov best k...,00493.1c5f59825f7a246187c137614fb1ea82
8,1,click simpli amateur like girl next door xxx f...,00012.381e4f512915109ba1e0853a7a8407b2
9,1,answer us unlist inform messag brought answer ...,00487.139a2f4e8edbbdd64441536308169d74


,CATEGORY,MESSAGE,FILE_NAME
0,1,homeown interest lowest point situat match hun...,00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,attent comput special packag norton systemwork...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,multi messag format nextpart content plain cha...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,import inform domain final avail general publi...,00210.050ffd105bd4e006771ee63cabc59978
4,1,bottom peopl month product sampl think precis ...,00033.9babb58d9298daa2963d4f514193d7d6
5,1,nextpart content plain charset content transfe...,00187.efd97ab2034b3384606e21db00014ecb
6,1,hello chines tradit emarket averag respons ban...,00500.85b72f09f6778a085dc8b6821965a76f
7,1,secur invest wealth without discov secret amer...,00493.1c5f59825f7a246187c137614fb1ea82
8,1,click simpli amateur first photo sneeki hidden...,00012.381e4f512915109ba1e0853a7a8407b2
9,1,answer unlist inform messag brought answer com...,00487.139a2f4e8edbbdd64441536308169d74


In [22]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jens.baetens3\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

Nu dat de tekst in zijn meest bruikbare vorm staat kan de "bag of words" bepaald worden van de dataset.
Dit gaat alle unieke woorden in de volledige dataset gaan oplijsten en komt overeen met de feature vector.
Er zijn dan twee manieren om de classificatie uit te voeren.
* [Multi-variate Bernoulli Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html): Kijk enkel of het woord in de bag-of-words voorkomt in de tekst of niet (0 of 1)
* [Multinomial Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html): Tel hoeveel keer elk woord in de bag-of-words voorkomt in de tekst. [CountVectorizer](https://www.educative.io/edpresso/countvectorizer-in-python) of [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html)

De TfidfTransformer is een speciale manier om de frequentie te berekenen, namelijk door het te vermenigvuldigen met het logaritme van het percentage van de observaties waarin het woord voorkomt.
Deze frequentie wordt de term frequency-inverse document frequency genoemd en zorgt ervoor dat woorden die veel voorkomen minder grote waarden krijgen dan woorden die zelden voorkomen. 

$\text{tfidf}_{i,j} = \text{tf}_{i,j} \times \log(\frac{N}{\text{df}_i})$

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect.fit(df.MESSAGE)
X_train = count_vect.transform(df.MESSAGE)
display(X_train[15, :40].toarray())

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer()
transformer.fit(X_train)
X_train_tfidf = transformer.transform(X_train)

Op deze verwerkte data kan dan een classifier getrained worden zoals LogisticRegression, SVM of naive bayes.
Merk op dat de bovenstaande vormen van naive bayes niet de enige zijn.
Een derde interessante versie is de [ComplementNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html) methode. 
Deze gaat vooral goed presteren wanneer de dataset sterk uit balans is.
Meer informatie over alle mogelijke implementaties van Naive Bayes vind je [hier](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.naive_bayes)

In [11]:
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, df.CATEGORY, test_size=0.2)

def test_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(accuracy_score(y_test, y_pred) * 100)

print("BernoulliNB")
test_model(BernoulliNB(), X_train, y_train,X_test,  y_test)

print("ComplementNB")
test_model(ComplementNB(), X_train, y_train,X_test,  y_test)

print("MultinomialNB")
test_model(MultinomialNB(), X_train, y_train,X_test,  y_test)

BernoulliNB
              precision    recall  f1-score   support

           0       0.71      1.00      0.83       769
           1       0.97      0.19      0.31       391

    accuracy                           0.72      1160
   macro avg       0.84      0.59      0.57      1160
weighted avg       0.80      0.72      0.65      1160

[[767   2]
 [318  73]]
72.41379310344827
ComplementNB
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       769
           1       0.99      0.90      0.94       391

    accuracy                           0.96      1160
   macro avg       0.97      0.95      0.96      1160
weighted avg       0.97      0.96      0.96      1160

[[767   2]
 [ 40 351]]
96.37931034482759
MultinomialNB
              precision    recall  f1-score   support

           0       0.89      1.00      0.94       769
           1       1.00      0.75      0.86       391

    accuracy                           0.91      1160
   macr